In [2]:
import numpy as np
import pandas as pd
import re
import os
import matplotlib as plt
import tkinter as tk
from tkinter import ttk
from tkinter import simpledialog
from tkinter import messagebox
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  NavigationToolbar2Tk)

#load app
def main():
    app = Application()

    def on_closing():
        if messagebox.askokcancel("Close", "Do you want to close?"):
            app.backupSave()

            app.window.destroy()

    app.window.protocol('WM_DELETE_WINDOW', on_closing)
    app.window.mainloop()

class Application():

    def __init__(self):
        #app window
        self.window = tk.Tk()
        self.cleanWindow = None
        self.manipulateWindow = None
        self.fileList = []
        self.cleanFileCombo = None
        self.manipulateFileCombo = None
        self.saveFileCombo = None
        self.inspectionsDF = None
        self.inventoryDF = None
        self.violationsDF = None
        self.titleStyle = None
        self.comboOption = ''

        self.createWidgets()

    #if any files have been open, save backup as json on close
    def backupSave(self):
        if(self.inspectionsDF is not None):
            self.inspectionsDF.to_json('InspectionsBackup' +'.json', orient='index', indent=4)
            if(self.inventoryDF is not None):
                self.inventoryDF.to_json('InventoryBackup' +'.json', orient='index', indent=4)
                if(self.violationsDF is not None):
                    self.violationsDF.to_json('ViolationsBackup' +'.json', orient='index', indent=4)

    #custom function to print formatted messages to text box
    def outputLog(self, message):
        self.outputText.insert('1.0', message + '\n')


    #read loaded files and check for errors, returns dataframe object for file
    def validateFile(self, file):
        fileType = os.path.splitext(file)[-1].lower()

        try:
            if(fileType == ".csv"):
                df = pd.read_csv(file)
            elif(fileType == ".json"):
                df = pd.read_json(file)
        except pd.errors.ParserError:
            self.outputLog('Error parsing data')
        except pd.errors.EmptyDataError:
            self.outputLog('File is empty')
        else:
            self.outputLog('Successful file validation')
            self.outputLog(f"{file} has been loaded")
            self.loadEntry.delete(0, tk.END)
            self.fileList.append(file)

            return df

    #Open file, notify user on successful load, clear entry field
    def loadFile(self):

        fileName = self.loadEntry.get()

        #create dataframes with valid file
        if(fileName == 'Inspections.csv'):
            self.inspectionsDF = self.validateFile(fileName) 
        elif(fileName == 'InspectionsBackup.json'):
            self.inspectionsDF = self.validateFile(fileName)  
        elif(fileName == 'Inventory.csv'):
            self.inventoryDF = self.validateFile(fileName)
        elif(fileName == 'InventoryBackup.json'):
            self.inventoryDF = self.validateFile(fileName)  
        elif(fileName == 'Violations.csv'):
            self.violationsDF = self.validateFile(fileName)
        elif(fileName == 'ViolationsBackup.json'):
            self.violationsDF = self.validateFile(fileName)  
        else:
            self.outputLog('Invalid file name')
            self.loadEntry.delete(0, tk.END)

        #update combobox
        self.updateComboBox(self.cleanFileCombo)

    #updates dropdown boxes to show imported files
    def updateComboBox(self, comboBox):
        comboBox['values'] = (self.fileList)
        comboBox.current()

    #widgets for clean window
    def openCleanWindow(self):

        self.cleanWindow = tk.Toplevel(self.window)
        self.cleanWindow.title('Clean options')
        self.cleanWindow.geometry('400x400')

        cleanTitle = ttk.Label(self.cleanWindow, text='Cleaning options', style='title.TLabel')
        cleanTitle.place(x=100, y=20)

        #drop null values
        dropNullTitle = ttk.Label(self.cleanWindow, text='Drop Null Data', style='subtitle.TLabel')
        dropNullTitle.place(x=30, y=80)
        dropNullRowsButton = ttk.Button(self.cleanWindow, text = 'Drop Null Rows', command=self.dropNullRows)
        dropNullRowsButton.place(height=35, width=100, x=30, y=110)
        dropNullColumnsButton = ttk.Button(self.cleanWindow, text = 'Drop Null Columns', command=self.dropNullColumns)
        dropNullColumnsButton.place(height=35, width=100, x=150, y=110)
        dropNullAllButton = ttk.Button(self.cleanWindow, text = 'Drop All Null', command=self.dropNullAll)
        dropNullAllButton.place(height=35, width=100, x=270, y=110)

        #Fill in null values
        fillNullTitle = ttk.Label(self.cleanWindow, text='Fill Null Data', style='subtitle.TLabel')
        fillNullTitle.place(x=30, y=180)
        fillNullButton = ttk.Button(self.cleanWindow, text = 'Fill Null', command=self.fillNull)
        fillNullButton.place(height=35, width=100, x=30, y=210)

        #fix data
        fixHeaders = ttk.Label(self.cleanWindow, text='Fix Data', style='subtitle.TLabel')
        fixHeaders.place(x=30, y=280)
        fixHeaderButton = ttk.Button(self.cleanWindow, text = 'Fix Headers', command=self.fixHeaderNames)
        fixHeaderButton.place(height=35, width=100, x=30, y=310)
        fixDateButton = ttk.Button(self.cleanWindow, text = 'Fix Date Format', command=self.fixDates)
        fixDateButton.place(height=35, width=100, x=150, y=310)

        #update combobox for save options
        # self.updateSaveCombobox()
        # self.updateManipulateCombobox()
        self.updateComboBox(self.saveFileCombo)
        self.updateComboBox(self.manipulateFileCombo)

    #set type and format
    def fixDates(self):

        self.inspectionsDF['ACTIVITY DATE'] = pd.to_datetime(self.inspectionsDF['ACTIVITY DATE'])
        self.inspectionsDF['ACTIVITY DATE'] = self.inspectionsDF['ACTIVITY DATE'].dt.strftime('%d/%m/%y')
        self.outputLog('New date format: D/M/Y')

    #remove extra spaces
    def fixHeaderNames(self):
        comboOption = self.cleanFileCombo.get()
        outputMessage = f'{comboOption} names fixed'

        def fixOperations(dataframe):
            dataframe.columns = dataframe.columns.str.lstrip()
            dataframe.columns = dataframe.columns.str.replace('  ', ' ')
            self.outputLog(dataframe.columns)

        if (comboOption == 'Inspections.csv'):
            fixOperations(self.inspectionsDF)
            self.outputLog(outputMessage)
        if (comboOption == 'Inventory.csv'):
            fixOperations(self.inventoryDF)
            self.outputLog(outputMessage)
        if (comboOption == 'Violations.csv'):
            fixOperations(self.violationsDF)
            self.outputLog(outputMessage)

    #drop if entire row/column is null
    def fillNull(self):

        comboOption = self.cleanFileCombo.get()
        fillValue = simpledialog.askstring(title='Fill Null', prompt='Enter fill value')
        outputMessage = f'{comboOption} null values replaced with {fillValue}\'s'

        if(comboOption == 'Inspections.csv'):
            self.inspectionsDF.fillna(fillValue, inplace=True)
            self.outputLog(outputMessage)
        if(comboOption == 'Inventory.csv'):
            self.inventoryDF.fillna(fillValue, inplace=True)
            self.outputLog(outputMessage)
        if(comboOption == 'Violations.csv'):
            self.violationsDF.fillna(fillValue, inplace=True)
            self.outputLog(outputMessage)

    #drop row if null value exists in 1 value
    def dropNullRows(self):

        comboOption = self.cleanFileCombo.get()
        outputMessage = f'{comboOption} null rows dropped'

        if(comboOption == 'Inspections.csv'):
            self.inspectionsDF.dropna()
            self.outputLog(outputMessage)
        elif(comboOption == 'Inventory.csv'):
            self.inventoryDF.dropna()
            self.outputLog(outputMessage)
        elif(comboOption == 'Violations.csv'):
            self.violationsDF.dropna()
            self.outputLog(outputMessage)

    #drop columns if null value exists in 1 value
    def dropNullColumns(self):

        comboOption = self.cleanFileCombo.get()
        outputMessage = f'{comboOption} null columns dropped'

        if(comboOption == 'Inspections.csv'):
            self.inspectionsDF = self.inspectionsDF.dropna(axis=1)
            self.outputLog(outputMessage)
        if(comboOption == 'Inventory.csv'):
            self.inventoryDF = self.inventoryDF.dropna(axis=1)
            self.outputLog(outputMessage)
        if(comboOption == 'Violations.csv'):
            self.violationsDF = self.violationsDF.dropna(axis=1)
            self.outputLog(outputMessage)

    #drop any rows/columns where all values are null
    def dropNullAll(self):

        comboOption = self.cleanFileCombo.get()
        outputMessage = f'{comboOption} all null rows/columns dropped'

        if(comboOption == 'Inspections.csv'):
            self.inspectionsDF.dropna(axis=0, how='all')
            self.inspectionsDF.dropna(axis=1, how='all')
            self.outputLog(outputMessage)
        if(comboOption == 'Inventory.csv'):
            self.inventoryDF.dropna(axis=0, how='all')
            self.inspectionsDF.dropna(axis=1, how='all')
            self.outputLog(outputMessage)
        if(comboOption == 'Violations.csv'):
            self.violationsDF.dropna(axis=0, how='all')
            self.inspectionsDF.dropna(axis=1, how='all')
            self.outputLog(outputMessage)

    #save data to JSON
    def saveData(self):

        popupTitle='Save File'
        popupPrompt='Enter file name (Exclude extension): '

        #check that combobox has options
        if(self.saveFileCombo != None):

            comboOption = self.saveFileCombo.get()
            if(comboOption == 'Inspections.csv'):
                saveFileName = simpledialog.askstring(title=popupTitle, prompt=popupPrompt)
                self.outputLog('Inspections.csv saved')
                self.inspectionsDF.to_json(saveFileName +'.json', orient='index', indent=4)

            elif(comboOption == 'Inventory.csv'):
                saveFileName = simpledialog.askstring(title=popupTitle, prompt=popupPrompt)
                self.outputLog('Inventory.csv saved')
                self.inventoryDF.to_json(saveFileName +'.json', orient='index', indent=4)

            elif(comboOption == 'Violations.csv'):
                saveFileName = simpledialog.askstring(title=popupTitle, prompt=popupPrompt)
                self.outputLog('Violations.csv saved')
                self.violationsDF.to_json(saveFileName +'.json', orient='index', indent=4)

    #window for managing manipulation window
    def openManipulateWindow(self):

        self.manipulateWindow = tk.Toplevel(self.window)
        self.manipulateWindow.title('Manipulate Data')
        self.manipulateWindow.geometry('400x500')

        manipulateTitle = ttk.Label(self.manipulateWindow, text='Manipulate Data', style='title.TLabel')
        manipulateTitle.place(x=100, y=20)

        #remove given values from chosen header
        removeDataTitle = ttk.Label(self.manipulateWindow, text='Remove data containing value', style='subtitle.TLabel')
        removeDataTitle.place(x=30, y=80)
        removeDataButton = ttk.Button(self.manipulateWindow, text = 'Remove', command=self.removeData)
        removeDataButton.place(height=35, width=100, x=270, y=120)

        indexNameLabel = ttk.Label(self.manipulateWindow, text='From Column: ')
        indexNameLabel.place(x=30, y=110)
        self.indexNameEntry = ttk.Entry(self.manipulateWindow, width=15)
        self.indexNameEntry.place(x=30, y=130)

        valueNameLabel = ttk.Label(self.manipulateWindow, text='Value: ')
        valueNameLabel.place(x=150, y=110)
        self.valueNameEntry = ttk.Entry(self.manipulateWindow, width=15)
        self.valueNameEntry.place(x=150, y=130)

        #remove data using regex and create new column
        extractDataTitle = ttk.Label(self.manipulateWindow, text='Extract bracket data to new column', style='subtitle.TLabel')
        extractDataTitle.place(x=30, y=170)
        extractDataButton = ttk.Button(self.manipulateWindow, text = 'Extract', command=self.extractData)
        extractDataButton.place(height=35, width=100, x=270, y=210)

        extractColumnLabel = ttk.Label(self.manipulateWindow, text='From Column: ')
        extractColumnLabel.place(x=30, y=200)
        self.extractColumnEntry = ttk.Entry(self.manipulateWindow, width=15)
        self.extractColumnEntry.place(x=30, y=220)

        newColumnNameLabel = ttk.Label(self.manipulateWindow, text='New Column: ')
        newColumnNameLabel.place(x=150, y=200)
        self.newColumnNameEntry = ttk.Entry(self.manipulateWindow, width=15)
        self.newColumnNameEntry.place(x=150, y=220)

        MMMTitle = ttk.Label(self.manipulateWindow, text='MMM: Inspections score p/year', style='subtitle.TLabel')
        MMMTitle.place(x=30, y=260)
        #mean
        calculateMeans = ttk.Button(self.manipulateWindow, text = 'Mean', command=self.calculateMean)
        calculateMeans.place(height=35, width=100, x=30, y=300)
        #median
        calculateMeans = ttk.Button(self.manipulateWindow, text = 'Median', command=self.calculateMedian)
        calculateMeans.place(height=35, width=100, x=150, y=300)
        #mode
        calculateMeansButton = ttk.Button(self.manipulateWindow, text = 'Mode', command=self.calculateMode)
        calculateMeansButton.place(height=35, width=100, x=270, y=300)

        violationsTitle = ttk.Label(self.manipulateWindow, text='Establishment Violations', style='subtitle.TLabel')
        violationsTitle.place(x=30, y=360)
        violationsButton = ttk.Button(self.manipulateWindow, text = 'Show Graph', command=self.generateViolationsGraph)
        violationsButton.place(height=35, width=100, x=30, y=400)

    #create graph for violations
    def generateViolationsGraph(self):

        options = self.manipulateFileCombo.get()
        if(options == 'Violations.csv'):

            self.violationsDF['COUNT'] = self.violationsDF.groupby(['VIOLATION CODE'])['VIOLATION CODE'].transform('count')

            violationCountDF = self.violationsDF[['VIOLATION CODE', 'COUNT']]
            violationCountDF = violationCountDF.set_index(['VIOLATION CODE'])
            violationCountDF = violationCountDF.drop_duplicates()

            self.graphWindow = tk.Toplevel(self.manipulateWindow)
            self.graphWindow.title('Graph Visualisation')
            self.graphWindow.geometry('1500x1000')

            #plot graph
            fig = Figure(figsize=(35, 25), dpi=100)
            ax1 = fig.add_subplot(111)
            violationCountDF.plot(kind='bar', legend=False, ax=ax1)

            #add graph to the GUI
            canvas = FigureCanvasTkAgg(fig, master=self.graphWindow)
            canvas.draw()
            canvas.get_tk_widget().pack
            toolbar = NavigationToolbar2Tk(canvas, self.graphWindow)
            toolbar.update()
            canvas.get_tk_widget().pack()

            self.outputText.insert('1.0', violationCountDF)

    #calculate mean for inspections score per year
    def calculateMean(self):

        options = self.manipulateFileCombo.get()
        if(options == 'Inspections.csv'):
            #means
            #by vendor seating
            self.inspectionsDF['YEAR'] = self.inspectionsDF['ACTIVITY DATE'].astype(str).str[-2:]

            mean = self.inspectionsDF['SCORE'].groupby([self.inspectionsDF['YEAR'], self.inspectionsDF['PE DESCRIPTION']]).mean()
            self.outputText.insert('1.0', mean)
            #by zip code
            mean = self.inspectionsDF['SCORE'].groupby([self.inspectionsDF['YEAR'], self.inspectionsDF['Zip Codes']]).mean()
            self.outputText.insert('1.0', mean)
        else:
            self.outputLog('Unable to calculate using current file selection')


    #calculate median for inspections score per year
    def calculateMedian(self):

        options = self.manipulateFileCombo.get()
        if(options == 'Inspections.csv'):
            #median
            #by vendor seating
            self.inspectionsDF['YEAR'] = self.inspectionsDF['ACTIVITY DATE'].astype(str).str[-2:]

            median = self.inspectionsDF['SCORE'].groupby([self.inspectionsDF['YEAR'], self.inspectionsDF['PE DESCRIPTION']]).median()
            self.outputText.insert('1.0', median)
            #by zip code
            median = self.inspectionsDF['SCORE'].groupby([self.inspectionsDF['YEAR'], self.inspectionsDF['Zip Codes']]).median()
            self.outputText.insert('1.0', median)
        else:
            self.outputLog('Unable to calculate using current file selection')

    #calculate mode for inspections score per year
    def calculateMode(self):

        options = self.manipulateFileCombo.get()
        if(options == 'Inspections.csv'):
            #mode
            #by vendor seating
            self.inspectionsDF['YEAR'] = self.inspectionsDF['ACTIVITY DATE'].astype(str).str[-2:]
            mode = self.inspectionsDF.groupby(['YEAR', 'PE DESCRIPTION'])['SCORE'].agg(pd.Series.mode).to_frame()
            self.outputText.insert('1.0', mode)

            #by zip code
            mode = self.inspectionsDF.groupby(['YEAR', 'Zip Codes'])['SCORE'].agg(pd.Series.mode).to_frame()
            self.outputText.insert('1.0', mode)
        else:
            self.outputLog('Unable to calculate using current file selection')

    #remove data values in a column and store in new col
    def extractData(self):

        comboOption = self.manipulateFileCombo.get()

        columnNameToExtract = self.extractColumnEntry.get()
        newColName = self.newColumnNameEntry.get()

        def extractOperations(dataframe):
            #dataframe[newColName] = dataframe[columnNameToExtract].str.extract('(\([\d]+-[\d]+\))', expand=False)
            #dataframe[columnNameToExtract] = dataframe[columnNameToExtract].str.replace('\([\d]+-[\d]+\)', '').str.replace('  ', ' ')

            dataframe[newColName] = dataframe[columnNameToExtract].str.extract('\((.*?)\)', expand=False)
            dataframe[columnNameToExtract] = dataframe[columnNameToExtract].str.replace('\((.*?)\)', '').str.replace('  ', ' ')

            print(dataframe)
            self.outputLog('Extract complete')
        
        if(comboOption == 'Inspections.csv'):
            extractOperations(self.inspectionsDF)
        elif(comboOption == 'Inventory.csv'):
            extractOperations(self.inventoryDF)
        elif(comboOption == 'Violations.csv'):
            extractOperations(self.violationsDF)

    #remove data from the dataframe
    def removeData(self):

        column = self.indexNameEntry.get()
        value = self.valueNameEntry.get()

        if column in self.inspectionsDF.columns or self.inventoryDF.columns or self.violationsDF.columns:
            comboOption = self.manipulateFileCombo.get()
            if(comboOption == 'Inspections.csv'):
                print('Inspections')
                self.inspectionsDF = self.inspectionsDF[self.inspectionsDF[column] != value].reset_index()
            if(comboOption == 'Inventory.csv'):
                print('Inventory')
                self.inventoryDF = self.inventoryDF[self.inventoryDF[column] != value].reset_index()
            if(comboOption == 'Violations.csv'):
                print('Violations')
                self.violationsDF = self.violationsDF[self.violationsDF[column] != value].reset_index()
        
    #empty text box
    def clearTextOutput(self):
        self.outputText.delete('1.0', tk.END)

    #create initial window layout
    def createWidgets(self):

        self.window.title("Data Analysis Tool")
        self.window.geometry("1000x500")

        #title
        titleLabel = ttk.Label(self.window, text='Data Analysis', style='title.TLabel')
        titleLabel.place(x=410, y=30)

        #create combo boxes
        self.cleanFileCombo = ttk.Combobox(self.window, width=15)
        self.cleanFileCombo.place(x=150, y=230)
        self.manipulateFileCombo = ttk.Combobox(self.window, width=15)
        self.manipulateFileCombo.place(x=150, y=320)
        self.saveFileCombo = ttk.Combobox(self.window, width=15)
        self.saveFileCombo.place(x=150, y=410)

        #load
        loadTitle = ttk.Label(self.window, text='Load', style='subtitle.TLabel')
        loadTitle.place(x=40, y=110)
        loadLabel = ttk.Label(self.window, text='Enter file name:')
        loadLabel.place(x=40, y=140)
        self.loadEntry = ttk.Entry(self.window, width=25)
        self.loadEntry.place(x=150, y=140)
        #load button
        loadButton = ttk.Button(self.window, text = 'Load', command=self.loadFile)
        loadButton.place(height=35, width=100, x=330, y=130)
        
        #clean
        cleanTitle = ttk.Label(self.window, text='Clean', style='subtitle.TLabel')
        cleanTitle.place(x=40, y=200)
        cleanLabel = ttk.Label(self.window, text='Select file')
        cleanLabel.place(x=40, y=230)
        #clean button
        cleanButton = ttk.Button(self.window, text = 'Clean', command=self.openCleanWindow)
        cleanButton.place(height=35, width=100, x=330, y=220)

        #manipulate
        manipulateTitle = ttk.Label(self.window, text='Manipulate', style='subtitle.TLabel')
        manipulateTitle.place(x=40, y=290)
        manipulateLabel = ttk.Label(self.window, text='Select file')
        manipulateLabel.place(x=40, y=320)
        #manipulate button
        manipulateButton = ttk.Button(self.window, text = 'Manipulate', command=self.openManipulateWindow)
        manipulateButton.place(height=35, width=100, x=330, y=310)

        #save
        saveTitle = ttk.Label(self.window, text='Save', style='subtitle.TLabel')
        saveTitle.place(x=40, y=380)
        saveLabel = ttk.Label(self.window, text='Select file')
        saveLabel.place(x=40, y=410)
        #save button
        saveButton = ttk.Button(self.window, text = 'Save', command=self.saveData)
        saveButton.place(height=35, width=100, x=330, y=400)

        #output field
        outputLabel = ttk.Label(self.window, text='Output:')
        outputLabel.place(x=475, y=100)
        self.outputText = tk.Text(self.window, height=20, width=58)
        self.outputText.place(x=475, y=120)
        outputScroll = tk.Scrollbar(command=self.outputText.yview)
        outputScroll.place(in_=self.outputText, relx=1.0, relheight=1.0, bordermode='outside')
        self.outputText['yscroll']=outputScroll.set

        #clear button
        clearButton = ttk.Button(self.window, text = 'Clear', command=self.clearTextOutput)
        clearButton.place(height=35, width=100, x=845, y=454)

        #styles
        #self.submitBtnStyle = ttk.Style().configure('submit.TButton', font=('', 10, 'bold'), background='DarkGrey', foreground='Black')
        self.titleStyle = ttk.Style().configure('title.TLabel', font=('', 18, 'bold'))
        self.subTitleStyle = ttk.Style().configure('subtitle.TLabel', font=('', 12, 'bold'))


#start prog
if __name__ == "__main__":
    main()
